In [3]:
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [4]:
loader = UnstructuredPDFLoader("../Downloads/Introduction to Genetics.pdf")

In [5]:
data = loader.load()

100%|████████████████████████████████████████████| 2/2 [00:00<00:00, 619.82it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 275.20it/s]
0it [00:00, ?it/s]
100%|███████████████████████████████████████████| 2/2 [00:00<00:00, 3453.52it/s]
0it [00:00, ?it/s]
100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 284.49it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 415.98it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]


In [6]:
print (f'There are {len(data)} document(s)')
print (f'There are {len(data[0].page_content)} characters in the document')

There are 1 document(s)
There are 511642 characters in the document


In [7]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(data)

In [8]:
print (f'There are {len(texts)} documents')

There are 702 documents


In [9]:
from langchain.vectorstores import Chroma, Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
import pinecone

In [10]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY =os.getenv.('OPENAI_API_KEY')
PINECONE_API_KEY = os.getenv.('PINECONE_API_KEY')
PINECONE_API_ENV = os.getenv.('PINECONE_API_ENV')

In [11]:
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

In [12]:
pinecone.init(
    api_key=PINECONE_API_KEY,
    environment=PINECONE_API_ENV,
)
index_name = "test"

In [13]:
search = Pinecone.from_texts([t.page_content for t in texts], embeddings, index_name=index_name)


In [14]:
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain

In [31]:
llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)
chain = load_qa_chain(llm, chain_type="stuff")

In [32]:
query = "What is RNA?"
docs = search.similarity_search(query, include_metadata=True)

In [55]:
response = chain.run(input_documents=docs, question=query)
print(response)

 RNA is a type of nucleic acid that is used as the genetic material in some viruses. It is similar to DNA, but is single-stranded and is typically used to create proteins.


In [56]:
from googletrans import Translator
translation = openai.Completion.create(
    engine="text-davinci-002",
    prompt=f"Translate this text into Hindi: {response}",
    max_tokens=2000,
    temperature=0.7,
)

translated_response = translation.choices[0].text
print(translated_response)



RNA एक प्रकार का न्यूक्लीय एसिड है जो कुछ वायरसों में जीनेटिक सामग्री के रूप में उपयोग किया जाता है। यह DNA के समान है लेकिन एक अक्षरी है और प्रायः प्रोटीन बनाने के लिए उपयोग किया जाता है।
